# Perceptron Algorithm

In [ ]:
import numpy as np

El algoritmo más sencillo de Perceptron:

In [ ]:
# x is dimension d by 1
# th is dimension d by 1
# th0 is dimension 1 by 1
# return 1 by 1 matrix of +1, 0, -1
def positive(x, th, th0):
   return np.sign(th.T@x + th0)


# Perceptron algorithm with offset.
# data is dimension d by n
# labels is dimension 1 by n
# T is a positive integer number of steps to run
def perceptron(data, labels, params = {}, hook = None):
    # if T not in params, default to 100
    T = params.get('T', 100)
    (d, n) = data.shape

    theta = np.zeros((d, 1)); theta_0 = np.zeros((1, 1))
    for t in range(T):
        for i in range(n):
            x = data[:,i:i+1]
            y = labels[:,i:i+1]
            if y * positive(x, theta, theta_0) <= 0.0:
                theta = theta + y * x
                theta_0 = theta_0 + y
                if hook: hook((theta, theta_0))
    return theta, theta_0

También podemos hacerlo pasar por el origen de coordenadas siempre.
Para que se pueda utilizar, el dataset D debe ser linealmente separable a través del origen, es decir:$\exists \theta$ tal que

$ y^{(i)}(\theta^T x^{(i)}\geq 0)$



Definimos el $\textbf{margin}$ de un data point $(x,y)$ como:$$y\cdot \frac{\theta^Tx}{||\theta ||}$$ Esto resulta ser unamuy buena medida de qué tan bien estamos (debería ser positivo si le atinamos). Y si estamos hablamos de un dataset, es el $i$ que tenga margin mínimo.$$$$

Con esto se establece el $\textbf{Perceptron Convergence Theorem}$: Si:
* $ \exists \theta^* | y^{(i)}\cdot \frac{\theta^{T*}x}{||\theta^* ||}\geq \gamma$               $ \forall i $
: Donde $\theta^*$ es el margin de $\theta^*$ en el data set
* $||x^{(i)}||\leq R$
: Es decir, los valores negativos se pueden encerrar en un circulo de radio real


Entonces el Perceptron va a tener máximo $\big(\frac{R}{\gamma}\big)^2$ errores

Para una solución más estable se puede utilizar el $\textbf{averaged perceptron}$:

In [ ]:
# x is dimension d by 1
# th is dimension d by 1
# th0 is dimension 1 by 1
# return 1 by 1 matrix of +1, 0, -1
def positive(x, th, th0):
   return np.sign(th.T@x + th0)

def averaged_perceptron(data, labels, params = {}, hook = None):
    T = params.get('T', 100)
    (d, n) = data.shape

    theta = np.zeros((d, 1)); theta_0 = np.zeros((1, 1))
    theta_sum = theta.copy() 
    theta_0_sum = theta_0.copy()
    for t in range(T):
        for i in range(n):
            x = data[:,i:i+1]
            y = labels[:,i:i+1]
            if y * positive(x, theta, theta_0) <= 0.0:
                theta = theta + y * x
                theta_0 = theta_0 + y
                if hook: hook((theta, theta_0))
            theta_sum = theta_sum + theta
            theta_0_sum = theta_0_sum + theta_0
    theta_avg = theta_sum / (T*n)
    theta_0_avg = theta_0_sum / (T*n)
    if hook: hook((theta_avg, theta_0_avg))
    return theta_avg, theta_0_avg
    

# Estrategias de Evaluación

In [ ]:
Captura de pantalla de 2021-01-02 14-12-38


def eval_classifier(learner, data_train, labels_train, data_test, labels_test):
    th, th0 = learner(data_train, labels_train)
    return score(data_test, labels_test, th, th0)/data_test.shape[1]


Toma un algoritmo de aprendizaje y una fuente de datos como entrada y ejecuta el algoritmo de aprendizaje varias veces, cada vez evaluando el clasificador resultante como se indicó anteriormente.

In [ ]:
def eval_learning_alg(learner, data_gen, n_train, n_test, it):
    score_sum = 0
    for i in range(it):
        data_train, labels_train = data_gen(n_train)
        data_test, labels_test = data_gen(n_test)
        score_sum += eval_classifier(learner, data_train, labels_train,
                                              data_test, labels_test)
    return score_sum/it      


La validación cruzada es una estrategia para evaluar un algoritmo de aprendizaje, utilizando un único conjunto de entrenamiento de tamaño $n$. La validación cruzada incluye un algoritmo de aprendizaje $L$, un conjunto de datos fijo $\mathcal {D}$ y un parámetro $k$. Ejecutará el algoritmo de aprendizaje $k$ veces diferentes, luego evaluará la precisión del clasificador resultante y, en última instancia, devolverá el promedio de las precisiones sobre cada una de las $k$ "corridas" de $L$. Está estructurado así:

In [1]:
def xval_learning_alg(learner, data, labels, k):
    s_data = np.array_split(data, k, axis=1)
    s_labels = np.array_split(labels, k, axis=1)

    score_sum = 0
    for i in range(k):
        data_train = np.concatenate(s_data[:i] + s_data[i+1:], axis=1)
        labels_train = np.concatenate(s_labels[:i] + s_labels[i+1:], axis=1)
        data_test = np.array(s_data[i])
        labels_test = np.array(s_labels[i])
        score_sum += eval_classifier(learner, data_train, labels_train,
                                              data_test, labels_test)
    return score_sum/k